In [0]:
import pyspark

In [0]:
import uuid

In [0]:
from pyspark.sql.functions import *
from datetime import datetime

In [0]:
control_df = spark.sql('select * from training.piyush.data_movement_control')
log_df = spark.sql('SELECT * FROM training.piyush.data_movement_logging')

In [0]:
control_df.display()

In [0]:
def data_movement(): 
    
    for row in control_df.collect():
        
        id = row[0]
        files = dbutils.fs.ls(row[2])
        # print(files)

        watermark_value = log_df.filter(log_df['table_id'] == id).agg(max(col('watermark_value')).alias ('latest_watermark_value')).collect()[0]['latest_watermark_value']
        # print(watermark_value)
        date = watermark_value
        # print(date)
        # print("filename", value.name)
        if date == None:
            date = datetime(1900, 1, 1)
        else:
            pass
        # print(modification_time, int(date.timestamp() * 1000))

        for value in files:
            print("value", value)
            filepath, modification_time = value.path, value.modificationTime
            # print(modification_time)
            # print(filepath)          
            if modification_time > int(date.timestamp() * 1000):

                dbutils.fs.cp(filepath, row[3] + '/' + value.name, recurse = True)
                # print(dbutils.fs.ls(filepath, row[3] + '/' + value.name, recurse = True))
                table_id = row[0]
                source_details = row[2] + value.name
                print(source_details)
                target_details = row[3] + value.name
                source_system = row[1]
                log_id = str(uuid.uuid4())   # generate unique ID
                # print(log_id)

                Status = 'SUCCESS'

                data = [(  log_id
                         , source_system
                         , table_id
                         , source_details
                         , target_details
                         , Status
                         )]

                df = spark.createDataFrame(data, ["log_id", "source_system", "table_id", "source_details",  "target_details", "Status"])

                df = df.withColumn('event_start_timestamp', current_timestamp())\
                       .withColumn('event_end_timestamp', current_timestamp())\
                       .withColumn('watermark_value', current_timestamp())

                df2 = df.select('log_id', 'source_system', 'table_id', 'source_details', 'target_details',      'event_start_timestamp', 'event_end_timestamp', 'Status', 'watermark_value')

                df2.write.mode('append').insertInto("training.piyush.data_movement_logging")
    
    
    return 'Success'
            







In [0]:
data_movement()

In [0]:
%sql
select * from training.piyush.data_movement_control

In [0]:
%sql
-- truncate table training.piyush.data_movement_logging


In [0]:
log_df.display()